#### Modelado de colas de espera en un hospital público de Guatemala
Integrantes: Stefano Aragoni, Luis Diego Santos, Carol Arevalo, Diego Perdomo



El sistema de salud pública en Guatemala, como en muchos otros países, enfrenta desafíos significativos. Uno de los más críticos es la prolongada espera que los pacientes deben soportar antes de recibir atención médica. Estas colas de espera pueden tener consecuencias graves, incluida una atención médica de menor calidad y una experiencia del paciente desfavorable. Este proyecto busca abordar este problema a través de técnicas avanzadas de modelado y simulación.

Hospital Roosvelt de Guatemala

- Cuenta con tres emergencias maternidad, adultos y pediatria
- Tiene capacidad para 950 pacientes en cama
- Se tine una capcidad de 50 pacientes en el area de emergencias
- Hay nueve camas para observación, 20 bancas para inhaloterapia y cuatro cuartos de shock.
- Diariamente atienden a un promedio de 2120 pacientes. 
- El 68.4% de los pacientes son consulta externa
- El 23.6% de los pacientes son de urgencias
- El 3.3% de los pacientes son hospitalizados
- El 2.4% de los pacientes son partos
- El 1.6% de los pacientes son cirugias selectivas
- El 0.4% de los pacientes entran a urgencias por traumas. 
- El 0.3% de los pacientes entran por obstetricia. 


Al entrar a urgencias:
1. Médicos evalúan si son casos que requieren atención de urgencia
2. Hay 4 clinicas de clasificacion. 
3. El 60% de los casos que entran a emergencias son *emfermedades comunes*. Estos son tratados en la clínica, con un tiempo de espera de 15 minutos.
4. Las *urgencias* son heridas, elevación de presión, deshidratación, o casos que merecen una atención mayor, estos pasan con un especialista, dijo el facultativo.
5. Cuando la persona llega con una herida de arma blanca o de fuego, con traumatismo, politraumatismo o complicaciones de enfermedades crónicas, se trata de una emergencia e ingresan cuanto antes para ser tratada en el área de Traumatología, Medicina interna o Cirugía.Estas personas entran a clinica de *claisificacion 1* directamente (desde ambulancias)
4. El tiempo de espera promedio era de 4 horas





#### Simulación del flujo de pacientes en urgencias en el Hospital Roosevelt de Guatemala.

##### 1. Definiciones y Parámetros Iniciales:

Empezamos por definir las constantes y parámetros basados en la información que has proporcionado:

In [180]:
import random

# Capacidades y parámetros del hospital
CAPACIDAD_EMERGENCIAS = 50
PACIENTES_POR_DIA = 2120
PORCENTAJE_URGENCIAS = 0.236

# Probabilidades para clasificación
PROB_ENFERMEDADES_COMUNES = 0.60
PROB_URGENCIAS_ESPECIALISTA = 1 - PROB_ENFERMEDADES_COMUNES

# Tiempo de espera
TIEMPO_ESPERA_COMUNES = 15 / 60  # Convertimos minutos a horas
TIEMPO_ESPERA_PROMEDIO = 4  # en horas


#### 2. Funciones de Simulación:

Las siguientes funciones nos ayudarán a simular la llegada de pacientes y su procesamiento en emergencias:

In [181]:
def clasificar_paciente():
    """
    Clasifica al paciente en base a probabilidades y regresa el tiempo de espera correspondiente.
    """
    rand_num = random.random()
    
    if rand_num <= PROB_ENFERMEDADES_COMUNES:
        return TIEMPO_ESPERA_COMUNES
    else:
        return TIEMPO_ESPERA_PROMEDIO

def simular_dia():
    """
    Simula un día en el hospital y regresa una lista con los tiempos de espera de los pacientes en urgencias.
    """
    num_pacientes_urgencias = int(PACIENTES_POR_DIA * PORCENTAJE_URGENCIAS)
    tiempos_espera = []

    for _ in range(num_pacientes_urgencias):
        tiempo_espera = clasificar_paciente()
        tiempos_espera.append(tiempo_espera)

    return tiempos_espera


3. Ejecución y Análisis:
Ahora, podemos ejecutar la simulación y obtener algunos resultados de interés:



In [182]:
def main():
    tiempos_espera = simular_dia()
    
    promedio_espera = sum(tiempos_espera) / len(tiempos_espera)
    pacientes_sobre_4_horas = sum(1 for t in tiempos_espera if t >= 4)

    print(f"Promedio de tiempo de espera: {promedio_espera:.2f} horas")
    print(f"Pacientes que esperaron más de 4 horas: {pacientes_sobre_4_horas}")

if __name__ == "__main__":
    main()


Promedio de tiempo de espera: 1.89 horas
Pacientes que esperaron más de 4 horas: 219


#### Simulación basada en agentes del flujo de pacientes en urgencias en el Hospital Roosevelt de Guatemala.

#### 1. Definición de Agentes:

Definiremos dos clases básicas: Paciente y Clinica.

In [196]:
from mesa import Agent
import random

class Paciente(Agent):
    ESPERANDO = 'entrando'
    SIENDO_ATENDIDO = 'siendo_atendido'
    FINALIZADO = 'finalizado'
    
    def __init__(self, unique_id, model, en_ambulancia=False):
        self.unique_id = unique_id
        self.model = model
        self.en_ambulancia = en_ambulancia
        self.tiempo_llegada = model.current_time  # Al crear un paciente, registramos el tiempo de llegada
        self.tiempo_espera = 0  # Inicializamos el tiempo de espera en 0
        self.estado = self.ESPERANDO  # Initialize the estado attribute here

    def esperar(self):
        self.tiempo_espera += 1

    def recibir_atencion(self):
        pass

    def finalizar(self):
        self.estado = self.FINALIZADO
        self.tiempo_espera = 0

    def step(self):
        self.esperar()
        
        if self.estado == self.ESPERANDO:
            # Si el paciente llegó en ambulancia
            if self.en_ambulancia:
                print("AMBULANCIA")
                clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaClasificacion1)
                if clinica_target:
                    print("CLINICA DISPONIBLE")
                    self.clinica_asignada = clinica_target
                    self.clinica_asignada.atender_paciente(self)
                    self.estado = 'clasificacion_1'
                    self.model.pacientes_clasificacion_1 += 1
                else:
                    print("CLINICA NO DISPONIBLE")
                    self.model.cola_clasificacion_1.append(self) 

            # Si no llegó en ambulancia, sigue la lógica anterior
            clinica_clasificadora_disponible = self.model.clinica_disponible(tipo_clinica=ClinicaClasificadora)
            if clinica_clasificadora_disponible:
                self.clinica_asignada = clinica_clasificadora_disponible
                self.clinica_asignada.atender_paciente(self)
                clasificacion = self.clinica_asignada.clasificar_paciente()

                if clasificacion == 'enfermedad_comun':
                    clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaEnfermedadesComunes)
                elif clasificacion == 'urgencia':
                    clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaUrgencias)
                elif clasificacion == 'clasificacion_1':
                    clinica_target = self.model.clinica_disponible(tipo_clinica=ClinicaClasificacion1)

                if clinica_target: # Si hay una clínica del tipo correcto disponible
                    self.clinica_asignada = clinica_target
                    self.clinica_asignada.atender_paciente(self)
                    self.estado = clasificacion

                    if clasificacion == 'enfermedad_comun':
                        self.model.pacientes_enfermedad_comun += 1
                    elif clasificacion == 'urgencia':
                        self.model.pacientes_urgencias += 1
                    elif clasificacion == 'clasificacion_1':
                        self.model.pacientes_clasificacion_1 += 1
                    
                else:  # Si no hay una clínica del tipo correcto disponible
                    if clasificacion == 'enfermedad_comun':
                        self.model.cola_enfermedad_comun.append(self)
                    elif clasificacion == 'urgencia':
                        self.model.cola_urgencias.append(self)
                    elif clasificacion == 'clasificacion_1':
                        self.model.cola_clasificacion_1.append(self)

        elif self.estado == self.SIENDO_ATENDIDO:
            self.recibir_atencion()
            if self.clinica_asignada and self.clinica_asignada.termino_atencion(self):
                self.finalizar()



class Clinica(Agent):
    def __init__(self, unique_id, model, capacidad=1):
        super().__init__(unique_id, model)
        self.capacidad = capacidad
        self.pacientes_actualmente = 0
        self.tiempo_espera_promedio = 0
        self.total_pacientes_atendidos = 0

    def atender_paciente(self, paciente):
        self.pacientes_actualmente += 1
        # Actualizamos el tiempo de espera promedio
        self.tiempo_espera_promedio = ((self.tiempo_espera_promedio * self.total_pacientes_atendidos) + paciente.tiempo_espera) / (self.total_pacientes_atendidos + 1)
        self.total_pacientes_atendidos += 1
        # Iniciar el tiempo de atención de esta clínica
        tiempo_de_atencion = self.tiempo_atencion()
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)

    def termino_atencion(self, paciente):
        # Por ahora, vamos a suponer que una vez que el paciente es atendido en una clínica, termina su atención inmediatamente.
        # Puedes ajustar esta lógica según tus necesidades.
        return True

    def esta_disponible(self):
        return self.pacientes_actualmente < self.capacidad

    def tiempo_atencion(self):
        pass

    def liberar(self):
        self.pacientes_actualmente -= 1
        if self.pacientes_actualmente < 0:
            self.pacientes_actualmente = 0
        self.check_cola_espera()
    
    def check_cola_espera(self):
        if isinstance(self, ClinicaEnfermedadesComunes) and self.model.cola_enfermedad_comun:
            paciente_en_espera = self.model.cola_enfermedad_comun.pop(0)
            paciente_en_espera.clinica_asignada = self
            self.atender_paciente(paciente_en_espera)
            paciente_en_espera.estado = 'enfermedad_comun'
        elif isinstance(self, ClinicaUrgencias) and self.model.cola_urgencias:
            paciente_en_espera = self.model.cola_urgencias.pop(0)
            paciente_en_espera.clinica_asignada = self
            self.atender_paciente(paciente_en_espera)
            paciente_en_espera.estado = 'urgencia'
        elif isinstance(self, ClinicaClasificacion1) and self.model.cola_clasificacion_1:
            paciente_en_espera = self.model.cola_clasificacion_1.pop(0)
            paciente_en_espera.clinica_asignada = self
            self.atender_paciente(paciente_en_espera)
            paciente_en_espera.estado = 'clasificacion_1'

class ClinicaEnfermedadesComunes(Clinica):
    def __init__(self, unique_id, model, capacidad=100):  # capacidad default es 10
        super().__init__(unique_id, model, capacidad)
    
    def tiempo_atencion(self):
        return random.randint(5, 30) 
    
    def atender_paciente(self, paciente):
        tiempo_espera_actual = self.model.current_time - paciente.tiempo_llegada
        paciente.tiempo_espera += tiempo_espera_actual
        self.model.tiempo_espera_enfermedad_comun += tiempo_espera_actual
        self.pacientes_actualmente += 1
        tiempo_de_atencion = self.tiempo_atencion() 
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)


class ClinicaUrgencias(Clinica):
    def __init__(self, unique_id, model, capacidad=50):  # capacidad default es 5
        super().__init__(unique_id, model, capacidad)
    
    def tiempo_atencion(self):
        # Aquí puedes definir el tiempo de espera promedio o alguna lógica adicional
        return random.randint(30, 180) 
    
    def atender_paciente(self, paciente):
        tiempo_espera_actual = self.model.current_time - paciente.tiempo_llegada
        paciente.tiempo_espera += tiempo_espera_actual
        self.model.tiempo_espera_urgencias += tiempo_espera_actual
        self.pacientes_actualmente += 1
        tiempo_de_atencion = self.tiempo_atencion()
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)

class ClinicaClasificacion1(Clinica):
    def __init__(self, unique_id, model, capacidad=20):  # capacidad default es 3
        super().__init__(unique_id, model, capacidad)
    
    def tiempo_atencion(self):
        return random.randint(10, 60) 
    
    def atender_paciente(self, paciente):
        tiempo_espera_actual = self.model.current_time - paciente.tiempo_llegada
        paciente.tiempo_espera += tiempo_espera_actual
        self.model.tiempo_espera_clasificacion_1 += tiempo_espera_actual
        self.pacientes_actualmente += 1
        tiempo_de_atencion = self.tiempo_atencion() 
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)
    

class ClinicaClasificadora(Clinica):
    def clasificar_paciente(self):
        r = random.random()
        if r < 0.60:
            return 'enfermedad_comun'
        else:
            return 'clasificacion_1'
        
    def tiempo_atencion(self):
        return random.randint(5, 30)
    
    def atender_paciente(self, paciente):
        tiempo_espera_actual = self.model.current_time - paciente.tiempo_llegada
        paciente.tiempo_espera += tiempo_espera_actual
        self.model.tiempo_espera_clasificadora += tiempo_espera_actual
        self.pacientes_actualmente += 1
        tiempo_de_atencion = self.tiempo_atencion()
        self.model.schedule_event(tipo="liberar_clinica", t=self.model.schedule.time + tiempo_de_atencion, clinica=self)

    

#### 2. Definición del Modelo:

El modelo controlará la lógica global de la simulación y gestionará todos los agentes.

In [197]:
class Event:
    def __init__(self, tipo, t, clinica):
        self.tipo = tipo
        self.t = t
        self.clinica = clinica


In [198]:
from mesa import Model
from mesa.time import RandomActivation

class HospitalModel(Model):
    def __init__(self, N, num_clinicas_clasificadoras=4, num_clinicas_comunes=100, num_clinicas_urgencias=50, num_clinicas_clasificacion1=5):
        self.num_pacientes = N
        self.upcoming_events = []
        self.schedule = RandomActivation(self)
        self.current_time = 0  # El tiempo actual de la simulación


        # Colas de espera
        self.cola_enfermedad_comun = []
        self.cola_urgencias = []
        self.cola_clasificacion_1 = []

        # Iniciar contadores y acumuladores de tiempo de espera
        self.pacientes_enfermedad_comun = 0
        self.pacientes_urgencias = 0
        self.pacientes_clasificacion_1 = 0
        self.tiempo_espera_enfermedad_comun = 0
        self.tiempo_espera_urgencias = 0
        self.tiempo_espera_clasificacion_1 = 0
        self.tiempo_espera_clasificadora = 0

        # Crear clínicas
        self.clinicas_clasificadoras = [ClinicaClasificadora(i, self) for i in range(num_clinicas_clasificadoras)]
        self.clinicas_comunes = [ClinicaEnfermedadesComunes(i, self) for i in range(num_clinicas_comunes)]
        self.clinicas_urgencias = [ClinicaUrgencias(i+len(self.clinicas_comunes), self) for i in range(num_clinicas_urgencias)]
        self.clinicas_clasificacion1 = [ClinicaClasificacion1(i+len(self.clinicas_comunes)+len(self.clinicas_urgencias), self) for i in range(num_clinicas_clasificacion1)]
        self.clinicas = self.clinicas_clasificadoras + self.clinicas_comunes + self.clinicas_urgencias + self.clinicas_clasificacion1


        # Crear pacientes
        for i in range(self.num_pacientes):
            paciente = Paciente(i, self, en_ambulancia=random.random() < 0.1)
            self.schedule.add(paciente)

    def reporte(self):
        print(f"Cantidad de pacientes que entraron: {self.num_pacientes}")
        print(f"Cantidad de pacientes de urgencias: {self.pacientes_urgencias}")
        print(f"Cantidad de pacientes de enfermedad común: {self.pacientes_enfermedad_comun}")
        print(f"Cantidad de pacientes de clasificación 1: {self.pacientes_clasificacion_1}")

        print(f"Promedio de tiempo de espera enfermedad común: {self.tiempo_espera_enfermedad_comun}")
        print(f"Promedio de tiempo de espera urgencias: {self.tiempo_espera_urgencias}")
        print(f"Promedio de tiempo de espera clasificación 1: {self.tiempo_espera_clasificacion_1}")
        print(f"Promedio de tiempo de espera clasificadora: {self.tiempo_espera_clasificadora}")



    def schedule_event(self, tipo, t, clinica):
        # Agregar evento a la lista
        event = Event(tipo, t, clinica)
        self.upcoming_events.append(event)
        # Ordenar la lista de eventos por tiempo para ejecutar el evento más próximo primero
        self.upcoming_events.sort(key=lambda e: e.t)

    def step(self):
        # Avanza un paso en la simulación
        self.schedule.step()

        # Manejar eventos
        while self.upcoming_events:
            # Tomar el próximo evento
            event = self.upcoming_events.pop(0)
            
            # Si el evento ocurre después del tiempo de finalización, terminamos la simulación
            if event.t > 1440:
                break

            # Avanzar el tiempo
            self.current_time = event.t

            # Ejecutar el evento
            self.execute_event(event)

    def clinica_disponible(self, tipo_clinica=None):
        for clinica in self.clinicas:
            if isinstance(clinica, tipo_clinica) and clinica.esta_disponible():
                return clinica
        return None
    
    def execute_event(self, event):
        if event.tipo == "liberar_clinica":
            event.clinica.liberar()


### 3. Ejecución:

Se crear una instancia del modelo y se ejecuta:

In [199]:
model = HospitalModel(2120)  

for i in range(1440):  
    model.step()

model.reporte()


Cantidad de pacientes que entraron: 2120
Cantidad de pacientes de urgencias: 0
Cantidad de pacientes de enfermedad común: 1161
Cantidad de pacientes de clasificación 1: 841
Promedio de tiempo de espera enfermedad común: 320523
Promedio de tiempo de espera urgencias: 0
Promedio de tiempo de espera clasificación 1: 207393
Promedio de tiempo de espera clasificadora: 525284
